In [1]:
import cv2
import os
import numpy as np
from PIL import Image
import pickle
import pandas as pd
from datetime import datetime

import time
from gtts import gTTS 
from IPython.display import Audio
import pyodbc
from playsound import playsound

In [11]:
#Código para generar el dataset
import cv2

web_cam = cv2.VideoCapture(0)

cascPath = "C:/Users/ALEXFER/Desktop/face_recognitionOpenCv2-master/Cascades/haarcascade_frontalface_default.xml"
faceCascade = cv2.CascadeClassifier(cascPath)

count = 0

while(True):
    _, imagen_marco = web_cam.read()

    grises = cv2.cvtColor(imagen_marco, cv2.COLOR_BGR2GRAY)

    rostro = faceCascade.detectMultiScale(grises, 1.5, 5)

    for(x,y,w,h) in rostro:
        cv2.rectangle(imagen_marco, (x,y), (x+w, y+h), (255,0,0), 4)
        count += 1
       
        cv2.imwrite("images/Alex_Velasque/Alex_"+str(count)+".jpg", grises[y:y+h, x:x+w])
        cv2.imshow("Creando Dataset", imagen_marco)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
    elif count >= 400:
        break


# Cuando todo está hecho, liberamos la captura
web_cam.release()
cv2.destroyAllWindows()

In [1]:
#Código para la conexión a BD

def tablaInsertar(nombre):    
    server='.'
    database='AsistenciaClaseIA'

    #Definir el string para la conexion
    conn = pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database)

    cursor=conn.cursor()
    conn.cursor().execute("insert into REGISTROASISTENCIA values (?,?)",(nombre,HOY) )
    conn.commit()
    cursor.close()
    

In [7]:
#Código para el entrenamiento
cascPath = "C:/Users/ALEXFER/Desktop/face_recognitionOpenCv2-master/Cascades/haarcascade_frontalface_alt2.xml"
faceCascade = cv2.CascadeClassifier(cascPath)

#reconocimiento con opencv
reconocimiento = cv2.face.LBPHFaceRecognizer_create()

BASE_DIR = os.path.dirname(os.path.abspath("__file__"))
image_dir = os.path.join(BASE_DIR,"images")


current_id = 0
etiquetas_id = {}
y_etiquetas = []
x_entrenamiento = []

for root, dirs, archivos in os.walk(image_dir):
    for archivo in archivos:
        if archivo.endswith("png") or archivo.endswith("jpg"):
            pathImagen = os.path.join(root,archivo)
            etiqueta = os.path.basename(root).replace(" ", "-")#.lower()
            #print(etiqueta,pathImagen)

            #Creando las etiquetas
            if not etiqueta in etiquetas_id:                
                etiquetas_id[etiqueta] = current_id
                current_id += 1            
            id_ = etiquetas_id[etiqueta]
            #print(etiquetas_id)

            pil_image = Image.open(pathImagen).convert("L")
            tamanio = (550,550)
            imagenFinal = pil_image.resize(tamanio, Image.ANTIALIAS)
            image_array = np.array(pil_image,"uint8")
            #print(image_array)

            rostros = faceCascade.detectMultiScale(image_array, 1.5, 5)

            for (x,y,w,h) in rostros:
                roi = image_array[y:y+h, x:x+w]
                x_entrenamiento.append(roi)
                y_etiquetas.append(id_)


#print(y_etiquetas)                
#print(x_entrenamiento)
with open("labels.pickle",'wb') as f:
    pickle.dump(etiquetas_id, f)

reconocimiento.train(x_entrenamiento, np.array(y_etiquetas))
reconocimiento.save("entrenamiento.yml")

In [18]:
#Código para el reconocimiento

# play the converted audio 

mytext =  "Fecha y hora de ingreso " + time.strftime("%c")
language = 'es'
mytext = mytext +  " Bienvenido "
mytext2 = mytext
presente = False


cascPath = "C:/Users/ALEXFER/Desktop/face_recognitionOpenCv2-master/Cascades/haarcascade_frontalface_alt2.xml"
faceCascade = cv2.CascadeClassifier(cascPath)

eyeCascade = cv2.CascadeClassifier("C:/Users/ALEXFER/Desktop/face_recognitionOpenCv2-master/Cascades/haarcascade_eye.xml")
smileCascade = cv2.CascadeClassifier("C:/Users/ALEXFER/Desktop/face_recognitionOpenCv2-master/Cascades/haarcascade_smile.xml")

reconocimiento = cv2.face.LBPHFaceRecognizer_create()
reconocimiento.read("entrenamiento.yml")

etiquetas = {"nombre_persona" : 1 }
with open("labels.pickle",'rb') as f:
    pre_etiquetas = pickle.load(f)
    etiquetas = { v:k for k,v in pre_etiquetas.items()}

web_cam = cv2.VideoCapture(0)

HOY = datetime.now().strftime('%Y-%m-%d')

cont = 0
nombre = ""
while True:
    if presente == True:
        playsound("welcome"+NOMBREWARD+".mp3")
        tablaInsertar(nombre)
        presente = False
        cont = 0
        NOMBREWARD = ""
        cv2.putText(marco, "COMPLETADO ;)", (x,y), font, 1, color, grosor, cv2.LINE_AA)
        #break
    # Capture el marco
    ret, marco = web_cam.read()
    grises = cv2.cvtColor(marco, cv2.COLOR_BGR2GRAY)    
    rostros = faceCascade.detectMultiScale(grises, 1.5, 5)
    nombre = ""
    # Dibujar un rectángulo alrededor de las rostros
    for (x, y, w, h) in rostros:
        #print(x,y,w,h)
        roi_gray = grises[y:y+h, x:x+w]
        roi_color = marco[y:y+h, x:x+w]
        # reconocimiento
        id_, conf = reconocimiento.predict(roi_gray)
        #nombre = ""
        if conf >= 4  and conf < 85:
            #print(id_)
            #print(etiquetas[id_])         
            font = cv2.FONT_HERSHEY_SIMPLEX            

            nombre = etiquetas[id_]
            #NOMBREWARD = etiquetas[id_]
            
            #if conf > 50:
             #   #print(conf)
              #  nombre = "Desconocido"

            color = (255,255,255)
            grosor = 2
            cv2.putText(marco, nombre, (x,y), font, 1, color, grosor, cv2.LINE_AA)
            if nombre != "":
                cont += 1
                
            if cont % 10 == 0:
                NOMBREWARD = nombre
                nombre = nombre.replace("_", " ")
                mytext = mytext2
                mytext = mytext +" " +nombre
                myobj = gTTS(text=mytext, lang=language, slow=False) 
                myobj.save("welcome"+ NOMBREWARD+ ".mp3") 
                presente = True
                #tablaInsertar(nombre)               
                #print(mytext)
                break
        img_item = "my-image.png"
        cv2.imwrite(img_item, roi_gray)
        
        cv2.rectangle(marco, (x, y), (x+w, y+h), (0, 255, 0), 2)

        rasgos = smileCascade.detectMultiScale(roi_gray)
        for(ex,ey,ew,eh) in rasgos:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 255, 0), 2)
    
    # Display resize del marco  
    marco_display = cv2.resize(marco, (1200, 650), interpolation = cv2.INTER_CUBIC)
    cv2.imshow('Detectando Rostros', marco_display)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Cuando todo está hecho, liberamos la captura
web_cam.release()
cv2.destroyAllWindows()
#playsound("welcome"+nombre+".mp3")

PermissionError: [Errno 13] Permission denied: 'welcomeJeff_Silva.mp3'

In [8]:
playsound("welcome.mp3")